In [316]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from decimal import Decimal

df_raw = pd.read_csv("db_dolly.txt", index_col=False, error_bad_lines=False)
df = df_raw[df_raw['doc']!="A Designar"]
df.head()
#materias = df.mat.unique()

,doc,mat,cuat,tstamp,asistencia,cumple_horarios,buen_trato,clase_organizada,claridad,fomenta_participacion,panorama_amplio,acepta_critica,responde_mails,comentarios
0,Garcia,6109,1Q2018,1533839977934,5,5,5,5,5,5,5,5,5,VW5hIGdlbmlhIHRvdGFsLCByZXNwb25kZSBhIHRvZG9zIG...
1,Travesaro,6109,1Q2018,1533839977936,5,5,5,5,5,5,5,5,5,NaN
2,Pastor,6109,1Q2018,1533839977937,5,5,5,5,4,5,5,5,5,NaN
3,Piva,6203,1Q2018,1533840118491,5,5,5,4,5,5,5,5,5,NaN
5,Di Luozzo,6203,1Q2018,1533840118496,5,5,5,3,3,3,5,5,5,NaN


# Calificación del docente

A fin de poder ordenar los docentes, para que los "mejores" aparezcan primero, es necesario obtener una métrica.
Voy a utilizar una norma 2 pero con coeficientes de ponderación dados consignados en la siguiente tabla.

In [317]:
pesos = {
    'asistencia': 1,
    'cumple_horarios': 1,
    'clase_organizada': .7,
    'claridad': .7,
    'buen_trato': 0.5,
    'acepta_critica': 0.5,
    'fomenta_participacion': 0.5,
    'responde_mails': 0.5,
    'panorama_amplio': 0.5
}
features = list(pesos.keys())
w = np.array(list(pesos.values()))
wn = w/np.sum(w)
np.sqrt(np.square(df[features].values) @ wn)

array([5.        , 5.        , 4.89205513, ..., 4.61941371, 4.86251661,
       4.12721435])

# Tabla de comparación

Tomando la media de cada feature, analizando para docente y materia por separado

In [318]:
# Para un docente particular
docentes = list(df['doc'].unique()) # pss... lista de docentes
df[df['doc']=='Acero'][features].mean()

asistencia               5.00
cumple_horarios          4.90
clase_organizada         4.75
claridad                 4.85
buen_trato               3.85
acepta_critica           4.00
fomenta_participacion    3.05
responde_mails           3.10
panorama_amplio          4.70
dtype: float64

In [319]:
# Para una materia en particular
puntajes = df[df['mat']==6103].groupby('doc')[features].mean()
puntajes.head()

,asistencia,cumple_horarios,clase_organizada,claridad,buen_trato,acepta_critica,fomenta_participacion,responde_mails,panorama_amplio
doc,,,,,,,,,
Acero,5.00,4.90,4.900000,4.900000,3.800000,4.200000,3.400000,2.800000,4.800000
Bergamini,5.00,5.00,5.000000,5.000000,5.000000,5.000000,4.000000,2.000000,5.000000
Boggi,3.75,3.75,3.000000,3.500000,4.000000,3.250000,3.750000,2.000000,3.750000
Canepa,3.75,4.00,4.250000,4.000000,4.750000,4.250000,3.750000,2.750000,3.750000
Chorny,4.50,4.50,4.666667,4.333333,4.666667,4.666667,4.333333,4.166667,4.666667


## Índice múltiple
Tendría que repetir el procedimiento para cada materia, pero si uso un indice múltiple (de la forma (materia,docente))
me ahorro todo ese trabajo.

In [320]:
counts = df.groupby(['mat','doc']).size().to_frame(name='respuestas')
grouped = df.groupby(['mat','doc'])[features].mean().join(counts)

grouped

asistencia  cumple_horarios  clase_organizada  \
mat  doc                                                                
6103 Acero                5.000000         4.900000          4.900000   
     Bergamini            5.000000         5.000000          5.000000   
     Boggi                3.750000         3.750000          3.000000   
     Canepa               3.750000         4.000000          4.250000   
     Chorny               4.500000         4.500000          4.666667   
     Comas                4.500000         4.500000          3.750000   
     Cristi               5.000000         5.000000          3.500000   
     De Rossi             4.000000         4.666667          4.833333   
     Endelli              5.000000         4.000000          2.000000   
     Estrin               3.000000         4.000000          1.000000   
     Fittipaldi           4.000000         4.000000          4.000000   
     Garcia               4.500000         4.250000          4.000000   
     Gigola               4.000000         5.000000          4.000000   
     Juarez               4.000000         4.000000          2.000000   
     Liberczuk            2.666667         2.666667          2.333333   
     Lopez                4.888889         4.777778          4.888889   
     Madariaga            5.000000         5.000000          3.500000   
     Martins              5.000000         5.000000          5.000000   
     Maulhardt            5.000000         4.916667          5.000000   
     Muszkats             4.666667         4.666667          4.666667   
     Oliva                4.000000         4.000000          3.000000   
     Palacios             5.000000         4.571429          4.428571   
     Palazón              5.000000         5.000000          3.000000   
     Porta                5.000000         5.000000          4.000000   
     Puebla               3.333333         3.666667          4.000000   
     Ramirez Oyhanarte    5.000000         4.500000          4.000000   
     Remesar              4.000000         4.000000          2.000000   
     Rosito               2.500000         3.000000          3.500000   
     Sarris               5.000000         5.000000          2.000000   
     Sassano              4.000000         4.666667          3.666667   
...                            ...              ...               ...   
9501 Lanzillotta          5.000000         5.000000          5.000000   
     Loyola Hidalgo       5.000000         5.000000          1.500000   
     Mayor                4.000000         3.000000          4.000000   
     Naon                 1.000000         1.000000          1.000000   
     Perez Berro          5.000000         5.000000          5.000000   
     Servetto             5.000000         5.000000          2.000000   
     Strobino             5.000000         3.000000          2.000000   
     Valverde             2.000000         4.000000          2.000000   
9503 Mendez               4.000000         5.000000          3.000000   
9511 Cardozo              4.250000         2.750000          3.750000   
     Kuhn                 5.000000         4.500000          2.500000   
     Moreno               5.000000         5.000000          5.000000   
     Roasio               5.000000         5.000000          4.666667   
     Sanz                 5.000000         5.000000          3.500000   
9513 Balzarotti           4.000000         5.000000          4.000000   
     Cavaliere            5.000000         5.000000          5.000000   
     Ezcurra              4.000000         5.000000          4.000000   
     Kowal                4.000000         5.000000          4.000000   
9514 Bianchi              3.000000         4.000000          4.000000   
     Essaya               5.000000         3.000000          5.000000   
     Gonzalez Busquin     4.000000         5.000000          5.000000   
     Guarna               1.000000         4.000000          3.000000   
     Juar

In [321]:
grouped.reset_index().to_json(orient='index')

'{"0":{"mat":6103,"doc":"Acero","asistencia":5.0,"cumple_horarios":4.9,"clase_organizada":4.9,"claridad":4.9,"buen_trato":3.8,"acepta_critica":4.2,"fomenta_participacion":3.4,"responde_mails":2.8,"panorama_amplio":4.8,"respuestas":10},"1":{"mat":6103,"doc":"Bergamini","asistencia":5.0,"cumple_horarios":5.0,"clase_organizada":5.0,"claridad":5.0,"buen_trato":5.0,"acepta_critica":5.0,"fomenta_participacion":4.0,"responde_mails":2.0,"panorama_amplio":5.0,"respuestas":1},"2":{"mat":6103,"doc":"Boggi","asistencia":3.75,"cumple_horarios":3.75,"clase_organizada":3.0,"claridad":3.5,"buen_trato":4.0,"acepta_critica":3.25,"fomenta_participacion":3.75,"responde_mails":2.0,"panorama_amplio":3.75,"respuestas":4},"3":{"mat":6103,"doc":"Canepa","asistencia":3.75,"cumple_horarios":4.0,"clase_organizada":4.25,"claridad":4.0,"buen_trato":4.75,"acepta_critica":4.25,"fomenta_participacion":3.75,"responde_mails":2.75,"panorama_amplio":3.75,"respuestas":4},"4":{"mat":6103,"doc":"Chorny","asistencia":4.5,"cum

In [324]:
def float_format(x):
    return str(round(Decimal(x),1))

html = grouped.to_html(float_format=float_format)

with open('datos.html', 'w') as f:
    print(html, file=f)